<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=4ab7fa3d6a883357cb6011521f7a5bed05f25538659591f0fd65687315e25a81
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-17 15:17:24
-------------------
qualified stocks: 88
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  2.18 L
Current:  1.43 C
-------------------
Today PnL: 97.78 K (0.69%)
Current PnL: -16.63 L (-11.19%)
CY Booked + Current PnL: -5.60 L (-3.77%)
-------------------
Total profit:  2.34 L
Total loss:  -18.97 L
-------------------
Total Booked + Current PnL: 21.38 L (17.57%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.74%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 82.53 L (57.89%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 9.82%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SBIN,863.00,-8.48,77.0,M-LC,4.43,227383.0,25904.0,1319.0,4.04,12.86,0.58,13.51,60.0,19.64,1.62,27.07,XY25,NTT,BANKS
5,APOLLOHOSP,8285.00,-13.20,59.0,H-LC,11.07,173536.0,17359.0,8729.0,0.91,11.12,5.03,16.71,22.0,1.99,1.24,30.49,X40N,BTT,HEALTHCARE
78,TTKPRESTIG,770.00,105.76,61.0,M-SC,4.41,88622.0,-12155.0,12248.0,0.65,-12.06,13.82,0.09,245.0,-0.99,0.63,16.43,OX40N,NTT,DURABLES
50,NATIONALUM,247.44,-43.66,65.0,H-MC,5.61,113028.0,12524.0,18118.0,-1.63,12.46,16.03,30.49,79.0,0.69,0.81,48.90,MH,ATH,METALS
57,RELIANCE,1533.00,-11.79,59.0,H-LC,5.60,220584.0,10218.0,18573.0,1.05,4.86,8.42,13.68,37.0,0.55,1.57,22.18,XY25,NTT,REFINERIES


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,IEX,219.00,-29.46,56.0,H-SC,17.30,201392.0,1438.0,95782.0,0.81,0.72,47.56,48.63,115.0,0.02,1.43,12.36,XR,NTT,MISC
35,ICICIPRULI,790.00,-21.22,42.0,H-MC,2.24,135611.0,-177.0,42934.0,-0.85,-0.13,31.66,31.48,107.0,-0.00,0.97,11.97,X40,ATH,INSURANCE
30,HAVELLS,2069.16,0.34,61.0,H-MC,1.79,248555.0,824.0,74243.0,-0.12,0.33,29.87,30.30,92.0,0.01,1.77,13.67,X40,ATH,ELECTRICAL
52,PGHH,17973.08,-30.81,52.0,H-MC,4.38,200250.0,-570.0,69347.0,0.76,-0.28,34.63,34.25,80.0,-0.01,1.43,4.90,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,VBL,671.64,-16.43,38.0,H-LC,5.59,300785.0,-15057.0,127713.0,-0.04,-4.77,42.46,35.67,5.0,-0.12,2.14,8.43,X40N,ATH,FMCG
6,ASIANPAINT,3465.66,-13.06,46.0,H-LC,8.18,219454.0,-32314.0,85521.0,-0.35,-12.83,38.97,21.13,27.0,-0.38,1.56,18.04,X40,ATH,PAINTS
42,ITC,452.00,-37.92,49.0,H-LC,2.91,200123.0,-15.0,20913.0,-0.82,-0.01,10.45,10.44,4.0,-0.00,1.43,5.56,X40,NTT,FMCG
44,JIOFIN,387.00,-0.10,50.0,H-LC,13.39,271416.0,4051.0,61394.0,0.08,1.52,22.62,24.48,48.0,0.07,1.93,57.32,XY24,BTT,FINANCE
20,CIPLA,1795.00,-19.78,51.0,H-LC,5.82,213747.0,9247.0,32169.0,0.77,4.52,15.05,20.25,10.0,0.29,1.52,13.65,X40N,BTT,PHARMA


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.20,-1.73,65.0,H-LC,0.99,244012.0,-2139.0,81354.0,1.52,-0.87,33.34,32.18,16.0,-0.03,1.74,34.86,X200,ATH,IT
42,ITC,452.00,-37.92,49.0,H-LC,2.91,200123.0,-15.0,20913.0,-0.82,-0.01,10.45,10.44,4.0,-0.00,1.43,5.56,X40,NTT,FMCG
51,NESTLEIND,1377.00,-8.48,56.0,H-LC,4.12,281549.0,16123.0,40656.0,-0.77,6.07,14.44,21.40,11.0,0.40,2.01,13.60,XY25,NTT,FMCG
84,VBL,671.64,-16.43,38.0,H-LC,5.59,300785.0,-15057.0,127713.0,-0.04,-4.77,42.46,35.67,5.0,-0.12,2.14,8.43,X40N,ATH,FMCG
57,RELIANCE,1533.00,-11.79,59.0,H-LC,5.60,220584.0,10218.0,18573.0,1.05,4.86,8.42,13.68,37.0,0.55,1.57,22.18,XY25,NTT,REFINERIES


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VAIBHAVGBL,521.00,64.74,71.0,H-SC,8.49,142951.0,-39824.0,150899.0,6.50,-21.79,105.56,60.77,125.0,-0.26,1.02,28.04,XR,NTT,JEWELLERY
70,SURYODAY,240.00,49.12,62.0,H-SC,2.79,136099.0,-42972.0,108702.0,5.10,-24.00,79.87,36.71,135.0,-0.40,0.97,35.04,XR,NTT,BANKS
17,CAMPUS,393.00,-22.75,65.0,M-SC,1.44,160970.0,-6805.0,62650.0,4.28,-4.06,38.92,33.28,210.0,-0.11,1.15,24.76,XY24,NTT,FOOTWEAR
63,SBIN,863.00,-8.48,77.0,M-LC,4.43,227383.0,25904.0,1319.0,4.04,12.86,0.58,13.51,60.0,19.64,1.62,27.07,XY25,NTT,BANKS
18,CAMS,5211.76,-1.17,62.0,H-SC,4.34,112364.0,10360.0,33563.0,3.97,10.16,29.87,43.06,122.0,0.31,0.80,29.01,X40N,ATH,MISC


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SFL,1287.00,36.64,45.0,M-SC,8.44,180316.0,-81902.0,151736.0,-4.08,-31.23,84.15,26.63,229.0,-0.54,1.28,12.11,XY24,NTT,MISC
37,INDIAMART,4850.92,-51.65,45.0,H-SC,12.09,135150.0,11814.0,121946.0,-2.27,9.58,90.23,108.45,119.0,0.10,0.96,34.07,AR,ATH,MISC
79,UJJIVANSFB,60.00,94.87,55.0,H-SC,12.41,123390.0,-19089.0,38609.0,-2.18,-13.40,31.29,13.70,163.0,-0.49,0.88,45.63,OX40N,NTT,BANKS
65,SHALBY,327.00,1062.68,65.0,M-SC,2.55,158219.0,-22924.0,68398.0,-1.90,-12.66,43.23,25.10,235.0,-0.34,1.13,26.80,XY24,NTT,HEALTHCARE
14,BERGEPAINT,680.00,-19.46,46.0,H-MC,6.00,216472.0,-10867.0,58924.0,-1.79,-4.78,27.22,21.14,106.0,-0.18,1.54,22.02,XY24,NTT,PAINTS


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,105.76,61.0,M-SC,4.41,88622.0,-12155.0,12248.0,0.65,-12.06,13.82,0.09,245.0,-0.99,0.63,16.43,OX40N,NTT,DURABLES
79,UJJIVANSFB,60.0,94.87,55.0,H-SC,12.41,123390.0,-19089.0,38609.0,-2.18,-13.40,31.29,13.70,163.0,-0.49,0.88,45.63,OX40N,NTT,BANKS
19,CERA,9475.0,-19.57,49.0,H-SC,2.14,146556.0,-29347.0,71373.0,0.84,-16.68,48.70,23.89,149.0,-0.41,1.04,26.57,OX40N,NTT,CERAMICS
45,KANSAINER,340.0,-67.42,61.0,H-SC,1.71,224550.0,-45117.0,81444.0,1.36,-16.73,36.27,13.47,138.0,-0.55,1.60,13.57,XY24,NTT,PAINTS
73,TATAELXSI,9161.0,-20.18,52.0,H-MC,7.82,107996.0,-23839.0,66061.0,-0.02,-18.08,61.17,32.03,98.0,-0.36,0.77,20.96,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4476.75,-25.15,59.0,H-LC,12.77,298112.0,-47844.0,122703.0,1.91,-13.83,41.16,21.64,1.0,-0.39,2.12,5.61,X40,ATH,IT
40,INFY,2275.00,-16.73,55.0,H-LC,8.76,324527.0,11639.0,160057.0,1.01,3.72,49.32,54.87,3.0,0.07,2.31,10.62,X40,BTT,IT
42,ITC,452.00,-37.92,49.0,H-LC,2.91,200123.0,-15.0,20913.0,-0.82,-0.01,10.45,10.44,4.0,-0.00,1.43,5.56,X40,NTT,FMCG
84,VBL,671.64,-16.43,38.0,H-LC,5.59,300785.0,-15057.0,127713.0,-0.04,-4.77,42.46,35.67,5.0,-0.12,2.14,8.43,X40N,ATH,FMCG
1,ABB,7934.00,-38.11,65.0,H-LC,11.09,258691.0,-2928.0,122128.0,0.95,-1.12,47.21,45.57,7.0,-0.02,1.84,10.08,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,45.15,66.0,M-MC,6.97,228940.0,3978.0,163669.0,0.12,1.77,71.49,74.53,192.0,0.02,1.63,34.59,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,12.54,62.0,H-MC,3.72,194200.0,29530.0,38432.0,2.80,17.93,19.79,41.28,89.0,0.77,1.38,27.35,X40N,ATH,AC
85,VOLTAS,1918.49,0.74,57.0,H-MC,2.69,211425.0,19683.0,76346.0,-0.44,10.27,36.11,50.08,99.0,0.26,1.51,17.76,XY25,ATH,AC
30,HAVELLS,2069.16,0.34,61.0,H-MC,1.79,248555.0,824.0,74243.0,-0.12,0.33,29.87,30.30,92.0,0.01,1.77,13.67,X40,ATH,ELECTRICAL
18,CAMS,5211.76,-1.17,62.0,H-SC,4.34,112364.0,10360.0,33563.0,3.97,10.16,29.87,43.06,122.0,0.31,0.80,29.01,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,247.44,-43.66,65.0,H-MC,5.61,113028.0,12524.0,18118.0,-1.63,12.46,16.03,30.49,79.0,0.69,0.81,48.90,MH,ATH,METALS
86,WHIRLPOOL,2270.00,-40.41,53.0,M-SC,0.68,100688.0,9190.0,69565.0,-0.08,10.04,69.09,86.07,223.0,0.13,0.72,46.22,XR,NTT,DURABLES
37,INDIAMART,4850.92,-51.65,45.0,H-SC,12.09,135150.0,11814.0,121946.0,-2.27,9.58,90.23,108.45,119.0,0.10,0.96,34.07,AR,ATH,MISC
87,WIPRO,420.00,-10.16,57.0,M-LC,5.55,157641.0,6696.0,102766.0,1.19,4.44,65.19,72.51,53.0,0.07,1.12,10.73,XR,NTT,IT
12,BANKINDIA,190.00,-29.19,66.0,H-MC,5.91,184052.0,4244.0,108167.0,1.42,2.36,58.77,62.52,88.0,0.04,1.31,37.25,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-41.65,52.0,L-SC,41.75,84960.0,-28589.0,68639.0,-0.38,-25.18,80.79,35.27,268.0,-0.42,0.61,107.94,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-4.84,46.0,H-SC,13.68,88640.0,-12323.0,106856.0,1.52,-12.21,120.55,93.63,148.0,-0.12,0.63,32.03,SR,ATH,CHEMICALS
49,MASFIN,398.61,-16.44,54.0,H-SC,12.90,94800.0,-3180.0,24781.0,-1.22,-3.25,26.14,22.05,152.0,-0.13,0.68,37.80,XR,ATH,FINANCE
50,NATIONALUM,247.44,-43.66,65.0,H-MC,5.61,113028.0,12524.0,18118.0,-1.63,12.46,16.03,30.49,79.0,0.69,0.81,48.90,MH,ATH,METALS
8,ATULAUTO,844.00,3851.35,64.0,M-SC,6.48,123204.0,-24164.0,75980.0,0.81,-16.40,61.67,35.16,236.0,-0.32,0.88,25.84,XY24,NTT,AUTO


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.0,3851.35,64.0,M-SC,6.48,123204.0,-24164.0,75980.0,0.81,-16.40,61.67,35.16,236.0,-0.32,0.88,25.84,XY24,NTT,AUTO
22,COFFEEDAY,80.0,-41.65,52.0,L-SC,41.75,84960.0,-28589.0,68639.0,-0.38,-25.18,80.79,35.27,268.0,-0.42,0.61,107.94,XR,NTT,HOTELS
27,GREENPANEL,537.0,237.86,67.0,M-SC,4.09,150890.0,-32188.0,111161.0,0.28,-17.58,73.67,43.14,230.0,-0.29,1.07,41.93,XY24,NTT,MISC
54,RAJESHEXPO,518.0,1889.90,67.0,L-SC,4.42,54296.0,-82881.0,82975.0,1.93,-60.42,152.82,0.07,267.0,-1.00,0.39,35.04,OX40N,NTT,JEWELLERY
76,TITAGARH,1548.0,0.53,68.0,H-SC,4.12,164946.0,-27069.0,105961.0,-0.61,-14.10,64.24,41.08,158.0,-0.26,1.17,39.77,XY24,NTT,ENGINEERING


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.48
1,25,42.77
2,50,73.20


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.30
LC    34.06
MC    22.63
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.46
X40      14.98
X40N     12.85
XR       11.44
XY25      9.65
AR        8.49
OX40N     8.03
X200      1.74
MH        1.68
X5K       1.42
SR        1.25
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.46
H-LC    25.88
H-MC    19.74
M-SC    13.31
M-LC     7.13
M-MC     2.59
L-SC     1.53
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.20,-13.31,75.10
FMCG,11.57,-4.28,40.76
FINANCE,9.48,-15.60,60.87
BANKS,7.76,-10.13,60.84
MISC,7.31,-11.80,73.98
PAINTS,5.70,-15.31,32.50
ELECTRICAL,5.58,-5.04,62.93
AC,3.60,4.06,28.44
INSURANCE,3.35,-7.22,44.00


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2847626.0,22
XR,1305909.0,14
AR,1302613.0,9
X40,746453.0,10
X40N,607952.0,10
OX40N,520822.0,10
XY25,397089.0,7
SR,245357.0,2
X5K,117682.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3414376.0,29
M-SC,1378719.0,17
H-LC,1241088.0,15
H-MC,1183603.0,15
M-LC,376766.0,5
M-MC,313397.0,2
L-SC,247902.0,3
L-MC,60490.0,1
L-LC,36270.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1028464.0      6
M-SC       XY24       797453.0      7
H-SC       AR         785239.0      5
           XR         755386.0      7
H-MC       XY24       491919.0      4
H-LC       X40        461769.0      5
           AR         355931.0      2
M-MC       XY24       313397.0      2
H-SC       X40N       288165.0      4
           OX40N      250129.0      4
           SR         245357.0      2
H-LC       X40N       221411.0      4
H-MC       X40        215290.0      4
           XY25       185672.0      2
L-SC       XR         164927.0      2
M-SC       AR         161443.0      2
M-LC       XY24       154999.0      2
M-SC       OX40N      121657.0      4
M-LC       X5K        117682.0      1
M-SC       XY25       114599.0      1
           XR         114173.0      2
H-MC       XR         108167.0      1
M-LC       XR         102766.0      1
H-MC       X40N        98376.0      2
L-SC       OX40N       82975.0      1
H-LC       X200        81354.0      1
M-SC       X40         69394.0      1
H-MC       OX40N       66061.0      1
H-SC       MH          61636.0      1
H-LC       XY24        61394.0      1
L-MC       XR          60490.0      1
H-LC       XY25        59229.0      2
L-LC       XY25        36270.0      1
H-MC       MH          18118.0      1
M-LC       XY25         1319.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
